In [1]:
import os
import pandas as pd
import numpy as np
import re
import pdfplumber
import string
import operator
import jieba
import jieba.analyse
# from collections import Counter
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction import text
# from zhon.hanzi import punctuation
import string

In [19]:
# 定义原始 PDF 文件夹路径和目标 TXT 文件夹路径
pdf_folder_path = r'G:\年报PDF02'
# txt_folder_path = r'C:\Users\Lin Weiran\Desktop\毕业论文\年报TXT'

In [3]:
data = pd.read_excel(r"C:\Users\Lin Weiran\Desktop\毕业论文\Data\年报数据cleaned.xlsx").sort_values(by = ['name','year'])
data

,filename,filepath,name,year
20080,GQY视讯：2013年年度报告,C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\GQY视讯：2...,GQY视讯,2013
8574,GQY视讯：2014年年度报告,C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\GQY视讯：2...,GQY视讯,2014
16485,GQY视讯：2015年年度报告,C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\GQY视讯：2...,GQY视讯,2015
5036,GQY视讯：2016年年度报告,C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\GQY视讯：2...,GQY视讯,2016
6199,GQY视讯：2017年年度报告,C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\GQY视讯：2...,GQY视讯,2017
...,...,...,...,...
11761,龙马环卫：2016年年度报告,C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\龙马环卫：20...,龙马环卫,2016
24892,龙马环卫：2017年年度报告（修订稿）,C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\龙马环卫：20...,龙马环卫,2017
19658,龙马环卫：2018年年度报告,C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\龙马环卫：20...,龙马环卫,2018
10813,龙马环卫：2019年年度报告,C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\龙马环卫：20...,龙马环卫,2019


In [20]:
files = os.listdir(pdf_folder_path)

## 年报文本分析

In [7]:
MDA = {2010:"董事会报告",2011:'董事会报告',2012:'董事会报告',2013:'董事会报告',2014:'董事会报告',2015:'管理层讨论与分析',2016:'经营情况讨论与分析',2017:'经营情况讨论与分析'
       ,2018:'经营情况讨论与分析',2019:'经营情况讨论与分析',2020:'经营情况讨论与分析',2021:'管理层讨论与分析',2022:'管理层讨论与分析',2023:'管理层讨论与分析'}

In [8]:
def clean_text(text):
    # 去除换行符、空格和标点符号
    cleaned_text = re.sub(r'[\n\s\W]+', ' ', text)
    cleaned = ''.join(cleaned_text.split())
    return cleaned

In [9]:
# 使用停用词
stop_words=[line.strip() for line in open('stop_words.txt','r',encoding='utf-8').readlines()]
jieba.load_userdict("userdict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LINWEI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.751 seconds.
Prefix dict has been built successfully.


In [10]:
# 数字化词典
# 人工智能
AI = ['人工智能','高端智能','工业智能','移动智能','智能控制','智能终端','智能移动','智能管理','智能工厂','智能物流','智能制造',
      '智能仓储','智能技术','智能设备','智能生产','智能网联','智能系统','智能化','自动控制','自动监测','自动监控','自动检测','自动生产',
      '数控','一体化','集成化','集成解决方案','集成控制','集成系统','工业云','未来工厂','智能故障诊断','生命周期管理','生产制造执行系统',
      '虚拟化','虚拟制造']
# 大数据
Data = ['数据管理','数据挖掘','数据网络','数据平台','数据中心','数据科学','数字控制','数字技术','数字通信','数字网络','数字智能',
        '数字终端','数字营销','数字化','大数据','云计算','云IT','云生态','云服务','云平台','区块链','物联网','机器学习']
# 信息化
Info = ['信息共享','信息管理','信息集成','信息软件','信息系统','信息网络','信息终端','信息中心','信息化','网络化','工业信息','工业通信']
# 商业模式
Comm = ['移动互联网','工业互联网','产业互联网','互联网解决方案','互联网技术','互联网思维','互联网行动','互联网业务','互联网移动','互联网应用',
        '互联网营销','互联网战略','互联网平台','互联网模式','互联网商业模式','互联网生态','电商','电子商务','Internet','互联网+',
        '线上线下','线上到线下','线上和线下','O2O','B2B','C2C','B2C','C2B']

In [37]:
df_words_count = pd.DataFrame(columns=['filename', 'AI', 'Data', 'Info', 'Comm','length'])

In [39]:
# 遍历 PDF 文件夹中的每个文件
for i in range(1359,1700):
    file_name = files[i]
    
    if file_name.replace('.PDF','') in data['filename'].values:
        pdf_file_path = os.path.join(pdf_folder_path,file_name)
    
        # 初始化容器
        word_count_AI = {word: 0 for word in AI}
        word_count_Data = {word: 0 for word in Data}
        word_count_Info = {word: 0 for word in Info}
        word_count_Comm = {word: 0 for word in Comm}



        # print(pdf_file_path)
        # year = data['year'].iloc[i]
        year = int(re.findall(r'(2\d{3})',file_name)[0])

        mda = MDA[year]
        # 打开 PDF 文件
        try:
            with pdfplumber.open(pdf_file_path) as pdf:
                text = ''

                # 查找目录页
                is_toc=False

                # 获取MD&A起止页码
                for page in pdf.pages:
                    if '目录' in page.extract_text()  and '三' in page.extract_text() and '四' in page.extract_text() and '七' in page.extract_text():
                        is_toc = True
                        toc = clean_text(page.extract_text())
                        break
                if is_toc:
                    match = re.search(rf'{mda}(\d+).*?(\d+)', toc)
                    if match:
                        start_page = int(match.group(1)) # 起始页码
                        end_page = int(match.group(2)) # 终止页码
                
                try:
                    
                    # 逐页读取文本并追加到 text 变量中
                    for page_num in range(start_page-1, end_page):
                        try:
                            page = pdf.pages[page_num]
                            text += page.extract_text() # MD&A原始文本（包含前后节部分）
                        except IndexError:
                            print(f'{i}页码有误')
                            break

                    # 提纯MD&A（删除前后节）
                    content = clean_text(text) # text文本清洗
                    # matches = re.findall(rf'第\w节({mda}.*?)第\w节', cleaned_text) # 匹配内容
                    # content = ''.join(matches)
                    length = len(content)

                    # print(f'md&a内容：{content}')

                    # 统计词频并存入dataframe
                    # 对文本进行分词
                    words = jieba.lcut(content)
                    # 统计word_dict中各词组在长文本中出现的频次
                    for word in words:
                        if word in AI:
                            word_count_AI[word] += 1
                        elif word in Data:
                            word_count_Data[word] += 1
                        elif word in Info:
                            word_count_Info[word] += 1
                        elif word in Comm:
                            word_count_Comm[word] += 1

                    # 输出统计结果
                    AI_words = sum(word_count_AI.values())
                    Data_words = sum(word_count_Data.values())
                    Info_words = sum(word_count_Info.values())
                    Comm_words = sum(word_count_Comm.values())

                    record = {'filename':file_name, 'AI':AI_words, 'Data':Data_words, 'Info':Info_words, 'Comm':Comm_words,'length':length}
                    df_words_count = df_words_count._append(record,ignore_index=True)
                    print(f"{i+1}个completed.")
                except NameError:
                    print(f'{i}页码有误')

        except FileNotFoundError:
            print('PDF not found')
    else:
        continue


1360个completed.
1361个completed.
1362个completed.
1363个completed.
1365个completed.
1366个completed.
1367个completed.
1368个completed.
1369个completed.
1370个completed.
1371个completed.
1372个completed.
1373个completed.
1374个completed.
1375个completed.
1376个completed.
1377个completed.
1378个completed.
1379个completed.
1380个completed.
1381个completed.
1383个completed.
1384个completed.
1385个completed.
1386个completed.
1387个completed.
1388个completed.
1391个completed.
1392个completed.
1393个completed.
1394个completed.
1395个completed.
1396个completed.
1397个completed.
1398个completed.
1399个completed.
1400个completed.
1402个completed.
1403个completed.
1404个completed.
1405个completed.
1407个completed.
1409个completed.
1410个completed.
1411个completed.
1412个completed.
1413个completed.
1414个completed.
1415个completed.
1417个completed.
1418个completed.
1419个completed.
1420个completed.
1421个completed.
1422个completed.
1423个completed.
1424个completed.
1425个completed.
1426个completed.
1427个completed.
1428个completed.
1429个completed.
1430个com

In [40]:
df_words_count

,filename,AI,Data,Info,Comm,length
0,冀东装备：2014年年度报告.PDF,1,0,0,0,11713
1,冀东装备：2014年年度报告.PDF,1,0,0,0,11713
2,冀东装备：2015年年度报告.PDF,3,1,0,0,9541
3,冀东装备：2017年年度报告.PDF,0,0,0,1,11620
4,冀东装备：2018年年度报告.PDF,3,0,0,0,10424
...,...,...,...,...,...,...
169,创世纪：2022年年度报告.PDF,26,10,8,0,29376
170,创力集团：2014年年度报告.PDF,14,7,13,0,17225
171,创力集团：2015年年度报告.PDF,10,1,1,0,12695
172,创力集团：2016年年度报告.PDF,2,0,2,0,15016


In [41]:
df_words_count.to_excel(r'C:\Users\Lin Weiran\Desktop\毕业论文\年报TXT\词频0106.xlsx')

## Test Field

#### 1. PDF 转TXT

In [57]:
pdf_file_path = data['filepath'].iloc[88]
file_name = data['filename'].iloc[88]
txt_file_path = os.path.join(txt_folder_path, file_name + '.txt')
# 打开 PDF 文件
with pdfplumber.open(pdf_file_path) as pdf:
    toc = ''
    text = ''
    
    # 查找目录页
    is_toc = False
    
    # 读取MD&A起止页码
    for page in pdf.pages:
        if '目录' in page.extract_text() and '第一节' in page.extract_text():
            is_toc = True
            break
    if is_toc:
        toc = clean_text(page.extract_text())
        match = re.search(r'管理层讨论与分析(\d+).*?(\d+)', toc)
        if match:
            start_page = int(match.group(1))
            end_page = int(match.group(2))
        
        for page_num in range(start_page-1, end_page):
            page = pdf.pages[page_num]
            text += page.extract_text()
        
    # 将文本写入到对应的 TXT 文件中
    with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)


print(f"Converted {pdf_file_path} to {txt_file_path}")

Converted C:\Users\Lin Weiran\Desktop\毕业论文\年报PDF\一汽富维：2022年年度报告.PDF to C:\Users\Lin Weiran\Desktop\毕业论文\年报TXT\一汽富维：2022年年度报告.txt


#### 解析目录

In [22]:
year = 2022
mda = '管理层讨论与分析'

In [31]:
match = re.search(rf'{mda}(\d+).*?(\d+)', toc)

In [32]:
if match:
    start_page = int(match.group(1))
    end_page = int(match.group(2))
print(start_page,end_page)

7 24


#### 2. 文本清洗
- 去除空格、换行符、标点

In [58]:
def clean_text(text):
    # 去除换行符、空格和标点符号
    cleaned_text = re.sub(r'[\n\s\W]+', ' ', text)
    cleaned = ''.join(cleaned_text.split())
    return cleaned

In [59]:
text = clean_text(text)

In [60]:
text

'2022年年度报告合计207875399133319991847882943393对公司根据公开发行证券的公司信息披露解释性公告第1号非经常性损益定义界定的非经常性损益项目以及把公开发行证券的公司信息披露解释性公告第1号非经常性损益中列举的非经常性损益项目界定为经常性损益的项目应说明原因适用不适用十一采用公允价值计量的项目适用不适用单位元币种人民币对当期利润的影响项目名称期初余额期末余额当期变动金额应收款项融资211344000044334400002320000000合计211344000044334400002320000000十二其他适用不适用第三节管理层讨论与分析一经营情况讨论与分析1不断强化运营管理深入推进降本增效多措并举深挖降本严控费用支出受供应链危机大宗商品价格上涨全球半导体短缺等不利因素都纷沓而至公司发挥集团优势有序开展协同采购节能物流等7大方面和11个维度的降本工作完成立项1581个全年实现降本65亿元有效消减不利影响持续推进绿色制造加强安全管理推进合规项目实施有效落实三同时管理要求提升项目安环质量通过开展精准节能诊断不断提升能源利用率能源成本持续降低充分识别减排机会强化污染治理过程管控完成重点源头减排项目46项实现减排污染物612吨双碳工作开展有序有力夯实运营管理基础全面提质增效开展质量自工程完结活动QCC运营活动以持续激励营造浓厚改善氛围攻克502个质量课题12家分子公司QCC参与率达成100实现质量管理与人员能力双提升及时准确获取客户信息快速彻底解决质量和交付问题提前识别并消除质量及交付绩效风险快速落实红旗客户质量要求建立并运行红旗质量特别管理体制绩效稳固保持赢得客户信任建立5大工艺生产效率数据对标机制识别差距寻找改善机会完成重点效率提升项目200项稳步提升生产效率深入推进数字化运营赋能公司发展建设经营分析数字化驾驶舱通过目视化支撑变化点管理驱动业务能力提升完成富维经营管理驾驶舱系统上线运行为公司经管层提供及时可靠的决策数据支撑聚焦痛点问题精准靶向实施实施6个制造数字化项目解决生产业务管理问题推动富维数字化进程2聚焦优质客户持续推进市场开源紧随一汽集团发展战略稳固一汽集团内市场份额快速响应客户产量调整全力保障供应一汽集团市场份额稳固其中富维安道拓富维东阳富维汽车镜及富维冲压件获得一汽红旗E系列内外饰及冲压项目富维东阳及富维海拉获得一汽大众外

#### 3. regex提取内容

In [65]:
# 定义正则表达式模式
pattern = r"第\w节(管理层讨论与分析一.*?)第\w节"

In [66]:
print(re.findall(pattern,text))

['管理层讨论与分析一经营情况讨论与分析1不断强化运营管理深入推进降本增效多措并举深挖降本严控费用支出受供应链危机大宗商品价格上涨全球半导体短缺等不利因素都纷沓而至公司发挥集团优势有序开展协同采购节能物流等7大方面和11个维度的降本工作完成立项1581个全年实现降本65亿元有效消减不利影响持续推进绿色制造加强安全管理推进合规项目实施有效落实三同时管理要求提升项目安环质量通过开展精准节能诊断不断提升能源利用率能源成本持续降低充分识别减排机会强化污染治理过程管控完成重点源头减排项目46项实现减排污染物612吨双碳工作开展有序有力夯实运营管理基础全面提质增效开展质量自工程完结活动QCC运营活动以持续激励营造浓厚改善氛围攻克502个质量课题12家分子公司QCC参与率达成100实现质量管理与人员能力双提升及时准确获取客户信息快速彻底解决质量和交付问题提前识别并消除质量及交付绩效风险快速落实红旗客户质量要求建立并运行红旗质量特别管理体制绩效稳固保持赢得客户信任建立5大工艺生产效率数据对标机制识别差距寻找改善机会完成重点效率提升项目200项稳步提升生产效率深入推进数字化运营赋能公司发展建设经营分析数字化驾驶舱通过目视化支撑变化点管理驱动业务能力提升完成富维经营管理驾驶舱系统上线运行为公司经管层提供及时可靠的决策数据支撑聚焦痛点问题精准靶向实施实施6个制造数字化项目解决生产业务管理问题推动富维数字化进程2聚焦优质客户持续推进市场开源紧随一汽集团发展战略稳固一汽集团内市场份额快速响应客户产量调整全力保障供应一汽集团市场份额稳固其中富维安道拓富维东阳富维汽车镜及富维冲压件获得一汽红旗E系列内外饰及冲压项目富维东阳及富维海拉获得一汽大众外饰及车灯项目富维伟世通及富维曼胡获得一汽丰田液晶仪表项目及空滤项目富维汽车镜获得奥迪B10项目积极开拓外部优质市场打造富维核心竞争力以外埠四大基地为依托积极拼抢外部优质市场优化市场结构全年新获取外部业务占比突破30富维安道拓获得广汽埃安项目富维东阳获得国际TOP级新能源企业项目及奥迪PPE保险杠充电口盖项目富维成都延锋彼欧获得吉利极氪项目富维汽车镜获得上海洛轲项目富维车轮富维冲压件富维本特勒及富维曼胡获得奔驰及长城汽车等企业共计9个项目外部市场开拓效果显著71882022年年度报告积极探索后市场业务全面开展养护类产品项目确定品牌战略以维品配品牌经营品牌市

In [67]:
# 使用正则表达式匹配所有符合条件的部分
matches = re.findall(pattern, text)

#### 4. 词频统计

In [132]:
# 使用停用词
stop_words=[line.strip() for line in open('stop_words.txt','r',encoding='utf-8').readlines()]

In [172]:
# 数字化词典
# 人工智能
AI = ['人工智能','高端智能','工业智能','移动智能','智能控制','智能终端','智能移动','智能管理','智能工厂','智能物流','智能制造',
      '智能仓储','智能技术','智能设备','智能生产','智能网联','智能系统','智能化','自动控制','自动监测','自动监控','自动检测','自动生产',
      '数控','一体化','集成化','集成解决方案','集成控制','集成系统','工业云','未来工厂','智能故障诊断','生命周期管理','生产制造执行系统',
      '虚拟化','虚拟制造']

In [173]:
# 大数据
Data = ['数据管理','数据挖掘','数据网络','数据平台','数据中心','数据科学','数字控制','数字技术','数字通信','数字网络','数字智能',
        '数字终端','数字营销','数字化','大数据','云计算','云IT','云生态','云服务','云平台','区块链','物联网','机器学习']

In [174]:
# 信息化
Info = ['信息共享','信息管理','信息集成','信息软件','信息系统','信息网络','信息终端','信息中心','信息化','网络化','工业信息','工业通信']

In [175]:
# 商业模式
Comm = ['移动互联网','工业互联网','产业互联网','互联网解决方案','互联网技术','互联网思维','互联网行动','互联网业务','互联网移动','互联网应用',
        '互联网营销','互联网战略','互联网平台','互联网模式','互联网商业模式','互联网生态','电商','电子商务','Internet','互联网+',
        '线上线下','线上到线下','线上和线下','O2O','B2B','C2C','B2C','C2B']

In [176]:
# Digital_dict = AI + Data + Info + Comm

In [177]:
jieba.load_userdict("userdict.txt")

In [178]:
word_count_AI = {word: 0 for word in AI}
word_count_Data = {word: 0 for word in Data}
word_count_Info = {word: 0 for word in Info}
word_count_Comm = {word: 0 for word in Comm}

In [179]:
# 对长文本进行分词
words = jieba.lcut(content)

In [181]:
# 统计word_dict中各词组在长文本中出现的频次
for word in words:
    if word in AI:
        word_count_AI[word] += 1
    elif word in Data:
        word_count_Data[word] += 1
    elif word in Info:
        word_count_Info[word] += 1
    elif word in Comm:
        word_count_Comm[word] += 1

# 输出统计结果
AI_words = sum(word_count_AI.values())
Data_words = sum(word_count_Data.values())
Info_words = sum(word_count_Info.values())
Comm_words = sum(word_count_Comm.values())
print(f'AI类词频：{AI_words}')
print(f'Data类词频：{Data_words}')
print(f'Info类词频：{Info_words}')
print(f'Comm类词频：{Comm_words}')

AI类词频：60
Data类词频：16
Info类词频：8
Comm类词频：20


In [182]:
length = len(content)

In [187]:
df_words_count = pd.DataFrame(columns=['filename', 'AI', 'Data', 'Info', 'Comm','length'])

In [190]:
record = {'filename':'file01', 'AI':AI_words, 'Data':Data_words, 'Info':Info_words, 'Comm':Comm_words,'length':length}
df_words_count._append(record,ignore_index=True)

,filename,AI,Data,Info,Comm,length
0,file01,60,16,8,20,17355
